In [20]:
import pandas as pd
import numpy as np
import xlrd, sqlite3, os, sys
from datetime import date

In [2]:
# Import Raw Data from excel files into dataFrames
df10=pd.read_excel('mlb_odds_2010.xlsx')
df10['Year'] = 2010
df11=pd.read_excel('mlb_odds_2011.xlsx')
df11['Year'] = 2011
df12=pd.read_excel('mlb_odds_2012.xlsx')
df12['Year'] = 2012
df13=pd.read_excel('mlb_odds_2013.xlsx')
df13['Year'] = 2013
df14=pd.read_excel('mlb_odds_2014.xlsx')
df14['Year'] = 2014
df15=pd.read_excel('mlb_odds_2015.xlsx')
df15['Year'] = 2015
df16=pd.read_excel('mlb_odds_2016.xlsx')
df16['Year'] = 2016
df17=pd.read_excel('mlb_odds_2017.xlsx')
df17['Year'] = 2017
df18=pd.read_excel('mlb_odds_2018.xlsx')
df18['Year'] = 2018
df19=pd.read_excel('mlb_odds_2019.xlsx')
df19['Year'] = 2019

In [3]:
# make list of dataFrames
dfList = [df10, df11, df12, df13, df14, df15, df16, df17, df18, df19]

In [4]:
# reformat data
for df in dfList:
    try:
        dfnew=df['RL'].str.split(pat="(", n=1, expand=True)
        dfnewer = dfnew[1].str.rstrip("\)")
        df['RunLine'] = dfnew[0].apply(float)
        df['RunLineOdds'] = dfnewer.apply(int)
        df.rename({"openou":"open_ou", "Unnamed: 18":"OpenOUOdds", "Unnamed: 19":"OpenOUOdds", "closeou":"close_ou", "closeOU":"close_ou", "close OU":"close_ou", "CloseOU":"close_ou", "Unnamed: 20":"CloseOUOdds", "Unnamed: 21":"CloseOUOdds"}, axis='columns', inplace=True)
    except:
        df.rename({"openou":"open_ou", "Unnamed: 18":"OpenOUOdds", "Unnamed: 19":"OpenOUOdds", "closeou":"close_ou", "closeOU":"close_ou", "close OU":"close_ou", "CloseOU":"close_ou", "Unnamed: 20":"CloseOUOdds", "Unnamed: 21":"CloseOUOdds"}, axis='columns', inplace=True)
    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
for df in dfList:
    colNew = []
    i = 0
    for index, row in df.iterrows():
        if index == 0:
            pass
        elif index % 2 == 0:
            i += 1
        colNew.append(i)
    df['gameno'] = colNew

In [5]:
def convt(num):
    if type(num) == str:
        num = float(num[0]) + .5
    return num
for df in dfList:
    df['close_ou'] = df['close_ou'].apply(convt)

In [6]:
# convert date format
for df in dfList:
    df['date'] = df['date'].astype(str) + df['year'].astype(str)
    pd.to_numeric(df['date'])
    df['date'] = pd.to_datetime(df['date'], errors='ignore', utc=True, format="%m%d%Y")

In [10]:
# create additional columns
for df in dfList:
    totRuns = []
    counter = 0
    try:
        for index, row in df.iterrows():
            if row.final == "NL":
                totRuns.append(row.final)
            elif index % 2 == 0:
                counter = 0
                counter += row.final
            else:
                counter += row.final
                totRuns.append(counter)
                totRuns.append(counter)
    except TypeError:
        print(row)
    df['totalruns'] = totRuns
    RunDif = []
    counter = 0
    try:
        for index, row in df.iterrows():
            if row.final == "NL":
                RunDif.append(row.final)
            elif index % 2 == 0:
                counter = 0
                team1 = row.final
            else:
                team2 = row.final
                dif = team1 - team2
                RunDif.append(dif)
                RunDif.append(-dif)
    except TypeError:
        print(row)
    df['rundif'] = RunDif
    winRL = []
    for index, row in df.iterrows():
        # favorite, win by more than 1.5 = True
        # favorite, win by less than 1.5 or lose = False
        # underdog, wins or loses by less than 1.5 = True
        # underdog, loses by more than 1.5 = True
        try:
            if row.runline < 0:
                if row.rundif > abs(row.runline):
                    winRL.append(True)
                else:
                    winRL.append(False)
            else:
                if row.rundif > 0 or abs(row.rundif) < row.runline:
                    winRL.append(True)
                else:
                    winRL.append(False)
        except AttributeError:
            winRL.append("NaN")
        except TypeError:
            winRL.append("NaN")
        except ValueError:
            winRL.append("NaN")
    df['winrl'] = winRL
    winTot = []
    for index, row in df.iterrows():
        try:
            if index % 2 == 0:
                if row.totalruns > row.close_ou:
                    winTot.append(True)
                else:
                    winTot.append(False)
            else:
                if row.totalruns > row.close_ou:
                    winTot.append(False)
                else:
                    winTot.append(True)
        except TypeError:
            winTot.append(True)
    df['wintotal'] = winTot

In [11]:
df1 = pd.concat(dfList, sort=True)

In [13]:
df1.drop(['year'], axis=1, inplace=True)
df1.drop(['rl'], axis=1, inplace=True)

In [33]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44656 entries, 0 to 285
Data columns (total 29 columns):
1st            44654 non-null float64
2nd            44654 non-null float64
3rd            44654 non-null float64
4th            44654 non-null float64
5th            44654 non-null object
6th            44652 non-null object
7th            44650 non-null object
8th            44642 non-null object
9th            44633 non-null object
close          44656 non-null float64
close_ou       44656 non-null float64
closeouodds    44656 non-null int64
date           44656 non-null datetime64[ns, UTC]
final          44656 non-null object
gameno         44656 non-null int64
open           44656 non-null int64
open_ou        44370 non-null float64
openou         286 non-null float64
openouodds     44656 non-null int64
pitcher        44613 non-null object
rot            44656 non-null int64
rundif         44656 non-null object
runline        24926 non-null float64
runlineodds    24926 non-nu

In [19]:
df1.reindex(columns=['date', '1st', '2nd', '3rd', '4th', '5th', '6th', '7th', '8th', '9th', 'close', 'close_ou', 'closeouodds', 'final', 'gameno', 'open', 'open_ou', 'openou', 'openouodds', 'pitcher', 'rot', 'rundif', 'runline', 'runlineodds', 'team', 'totalruns', 'vh', 'winrl', 'wintotal'])

,date,1st,2nd,3rd,4th,5th,6th,7th,8th,9th,...,pitcher,rot,rundif,runline,runlineodds,team,totalruns,vh,winrl,wintotal
0,2010-04-04 00:00:00+00:00,0.0,2.0,0.0,3.0,0,0,2,0,0,...,CSABATHIA-L,931,-2,NaN,NaN,NYY,16,V,NaN,True
1,2010-04-04 00:00:00+00:00,0.0,1.0,0.0,0.0,1,3,3,1,x,...,JBECKETT-R,932,2,NaN,NaN,BOS,16,H,NaN,False
2,2010-04-05 00:00:00+00:00,0.0,0.0,0.0,5.0,0,2,4,0,0,...,RHALLADAY-R,901,10,NaN,NaN,PHI,12,V,NaN,True
3,2010-04-05 00:00:00+00:00,1.0,0.0,0.0,0.0,0,0,0,0,0,...,JLANNAN-L,902,-10,NaN,NaN,WAS,12,H,NaN,False
4,2010-04-05 00:00:00+00:00,0.0,0.0,0.0,0.0,0,1,0,0,0,...,JJOHNSON-R,903,-6,NaN,NaN,MIA,8,V,NaN,True
5,2010-04-05 00:00:00+00:00,2.0,0.0,0.0,0.0,0,4,1,0,x,...,JOSANTANA-L,904,6,NaN,NaN,NYM,8,H,NaN,False
6,2010-04-05 00:00:00+00:00,1.0,0.0,0.0,2.0,1,0,2,0,5,...,CARPENTER-R,905,5,NaN,NaN,STL,17,V,NaN,True
7,2010-04-05 00:00:00+00:00,0.0,0.0,0.0,2.0,0,0,1,1,2,...,AHARANG-R,906,-5,NaN,NaN,CIN,17,H,NaN,False
8,2010-04-05 00:00:00+00:00,2.0,0.0,0.0,0.0,0,0,3,0,0,...,VPADILLA-R,907,-6,NaN,NaN,LOS,16,V,NaN,True
9,2010-04-05 00:00:00+00:00,2.0,0.0,1.0,0.0,5,0,0,3,x,...,ZDUKE-L,908,6,NaN,NaN,PIT,16,H,NaN,False


In [37]:
DBFILENAME = os.path.join(os.path.abspath(""), 'baseball.db')
with sqlite3.connect(DBFILENAME) as conn:
    df1.to_sql('games', con=conn, if_exists='replace', dtype={'date': 'DATETIME', '1st': 'INTEGER', '2nd': 'INTEGER', '3rd': "INTEGER", '4th':'INTEGER', '5th': 'INTEGER', '6th': 'INTEGER', '7th':'INTEGER', '8th': 'INTEGER', '9th': 'INTEGER', 'close': 'REAL', 'close_ou': 'REAL', 'closeouodds': 'REAL', 'final': 'REAL', 'gameno': 'REAL', 'open':"REAL", 'open_ou': 'REAL', 'openou':'REAL', 'openouodds': 'REAL', 'pitcher': 'TEXT', 'rot':'INTEGER', 'rundif': 'REAL', 'runline':'REAL', 'runlineodds': 'REAL', 'team': 'TEXT', 'totalruns': 'INTEGER', 'vh': 'TEXT', 'winrl':'BOOLEAN', 'wintotal': 'BOOLEAN'})

In [38]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44656 entries, 0 to 285
Data columns (total 29 columns):
1st            44654 non-null float64
2nd            44654 non-null float64
3rd            44654 non-null float64
4th            44654 non-null float64
5th            44654 non-null object
6th            44652 non-null object
7th            44650 non-null object
8th            44642 non-null object
9th            44633 non-null object
close          44656 non-null float64
close_ou       44656 non-null float64
closeouodds    44656 non-null int64
date           44656 non-null datetime64[ns, UTC]
final          44656 non-null object
gameno         44656 non-null int64
open           44656 non-null int64
open_ou        44370 non-null float64
openou         286 non-null float64
openouodds     44656 non-null int64
pitcher        44613 non-null object
rot            44656 non-null int64
rundif         44656 non-null object
runline        24926 non-null float64
runlineodds    24926 non-nu